In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Option 1: Add project root to sys.path
import sys
from pathlib import Path

# Go up two levels from notebooks/centralized to reach project root
project_root = str(Path().absolute().parent.parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Now you can import from src normally
from src.common.client_utils import load_femnist_dataset

/home/lcm76/.conda/envs/fl-project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-18 16:19:39,861	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
from pathlib import Path
from typing import Any
from logging import INFO, DEBUG
import json
import os
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from scipy.signal import medfilt
from flwr.common import log, ndarrays_to_parameters
import matplotlib.pyplot as plt
import math

from src.common.client_utils import (
    load_femnist_dataset,
    get_network_generator_cnn as get_network_generator,
    get_device,
    get_model_parameters,
    aggregate_weighted_average,
)


from src.flwr_core import (
    set_all_seeds,
    get_paths,
    decompress_dataset,
    get_flower_client_generator,
    sample_random_clients,
    get_federated_evaluation_function,
    create_iid_partition
)

from src.estimate import (
    compute_critical_batch,
)

from src.experiments_simulation import (
    run_simulation,
    centralized_experiment,
)

from src.utils import get_centralized_acc_from_hist

PathType = Path | str | None

In [4]:
set_all_seeds()

PATHS = get_paths()

HOME_DIR = PATHS["home_dir"]
DATASET_DIR = PATHS["dataset_dir"]
DATA_DIR = PATHS["data_dir"]
CENTRALIZED_PARTITION = PATHS["centralized_partition"]
CENTRALIZED_MAPPING = PATHS["centralized_mapping"]
FEDERATED_PARTITION = PATHS["federated_partition"]
FEDERATED_IID_PARTITION = PATHS["iid_partition"]

# extract dataset from tar.gz
decompress_dataset(PATHS)

In [5]:
max_clients = 1000
create_iid_partition(PATHS, num_clients=max_clients)

Distributing data to client 0
Distributing data to client 1
Distributing data to client 2
Distributing data to client 3
Distributing data to client 4
Distributing data to client 5
Distributing data to client 6
Distributing data to client 7
Distributing data to client 8
Distributing data to client 9
Distributing data to client 10
Distributing data to client 11
Distributing data to client 12
Distributing data to client 13
Distributing data to client 14
Distributing data to client 15
Distributing data to client 16
Distributing data to client 17
Distributing data to client 18
Distributing data to client 19
Distributing data to client 20
Distributing data to client 21
Distributing data to client 22
Distributing data to client 23
Distributing data to client 24
Distributing data to client 25
Distributing data to client 26
Distributing data to client 27
Distributing data to client 28
Distributing data to client 29
Distributing data to client 30
Distributing data to client 31
Distributing data 

In [9]:
NETWORK_GENERATOR = get_network_generator()
SEED_NET = NETWORK_GENERATOR()
SEED_MODEL_PARAMS = get_model_parameters(SEED_NET)
CID_CLIENT_GENERATOR = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_IID_PARTITION)

In [11]:
# FL experiments
experiment_batch_sizes = [32, 64, 128, 256, 512]
cohort_sizes = [5, 10, 20, 50, 75, 100, 150]


# Federated configuration dictionary
federated_train_config = {
    "epochs": 10,
    "batch_size": 32,
    "client_learning_rate": 0.01,
    "weight_decay": 0,
    "num_workers": 0,
    "max_batches": 100,
}

federated_test_config: dict[str, Any] = {
    "batch_size": 32,
    "num_workers": 0,
    "max_batches": 100,
}

num_rounds = 10
num_total_clients = 100
num_evaluate_clients = 0
num_clients_per_round = 10

initial_parameters = ndarrays_to_parameters(SEED_MODEL_PARAMS)

federated_evaluation_function = get_federated_evaluation_function(
    batch_size=federated_test_config["batch_size"],
    num_workers=federated_test_config["num_workers"],
    model_generator=NETWORK_GENERATOR,
    criterion=nn.CrossEntropyLoss(),
    max_batches=None if "max_batches" not in federated_test_config else federated_test_config["max_batches"],
)

server_learning_rate = 1.0
server_momentum = 0.0
accept_failures = False


CID_CLIENT_GENERATOR = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_IID_PARTITION)

list_of_ids = sample_random_clients(
    num_total_clients, federated_train_config["batch_size"],
    CID_CLIENT_GENERATOR, max_clients=max_clients,
)

federated_client_generator = (
    get_flower_client_generator(
        NETWORK_GENERATOR, FEDERATED_IID_PARTITION, lambda seq_id: list_of_ids[seq_id]
    )
)

INFO flwr 2025-03-18 16:12:06,962 | flwr_core.py:107 | cid: 632
INFO flwr 2025-03-18 16:12:06,968 | flwr_core.py:107 | cid: 947
INFO flwr 2025-03-18 16:12:06,973 | flwr_core.py:107 | cid: 546
INFO flwr 2025-03-18 16:12:06,978 | flwr_core.py:107 | cid: 726
INFO flwr 2025-03-18 16:12:06,982 | flwr_core.py:107 | cid: 374
INFO flwr 2025-03-18 16:12:06,987 | flwr_core.py:107 | cid: 584
INFO flwr 2025-03-18 16:12:06,992 | flwr_core.py:107 | cid: 599
INFO flwr 2025-03-18 16:12:06,996 | flwr_core.py:107 | cid: 749
INFO flwr 2025-03-18 16:12:07,000 | flwr_core.py:107 | cid: 169
INFO flwr 2025-03-18 16:12:07,004 | flwr_core.py:107 | cid: 793
INFO flwr 2025-03-18 16:12:07,008 | flwr_core.py:107 | cid: 844
INFO flwr 2025-03-18 16:12:07,012 | flwr_core.py:107 | cid: 340
INFO flwr 2025-03-18 16:12:07,016 | flwr_core.py:107 | cid: 392
INFO flwr 2025-03-18 16:12:07,021 | flwr_core.py:107 | cid: 650
INFO flwr 2025-03-18 16:12:07,026 | flwr_core.py:107 | cid: 808
INFO flwr 2025-03-18 16:12:07,031 | flwr

In [12]:
import pickle

def save_experiment(save_file_name, batch_size, parameters_for_each_round, hist):
    results_dict = {
        'batch_size': batch_size,
        'parameters_for_each_round': parameters_for_each_round,
        'history': hist
    }
    
    with open(save_file_name, 'wb') as f:  # Note: 'wb' for binary write mode
        pickle.dump(results_dict, f)

def load_experiment(file_name):
    with open(file_name, 'rb') as f:  # Note: 'rb' for binary read mode
        results_dict = pickle.load(f)
    
    return (
        results_dict['batch_size'],
        results_dict['parameters_for_each_round'],
        results_dict['history'],
    )

In [ ]:
total_batch_results = []

experiment_batch_sizes = [16, 32, 64, 128, 256]

for batch_size in experiment_batch_sizes:
    train_cfg = federated_train_config.copy()
    train_cfg["batch_size"] = batch_size
    ratio = np.sqrt(batch_size / 256)
    learning_rate = ratio * 0.01
    train_cfg["client_learning_rate"] = learning_rate # Same as centralized, but should be lower for FL

    test_cfg = federated_test_config.copy()
    test_cfg["batch_size"] = batch_size

    local_list_of_ids = sample_random_clients(num_total_clients, train_cfg["batch_size"], CID_CLIENT_GENERATOR, max_clients=max_clients)
    local_federated_client_generator = get_flower_client_generator(NETWORK_GENERATOR, FEDERATED_IID_PARTITION, lambda seq_id: local_list_of_ids[seq_id])

    parameters_for_each_round, hist = run_simulation(
        num_rounds = num_rounds,
        num_total_clients = num_total_clients,
        num_clients_per_round = num_clients_per_round,
        num_evaluate_clients = num_evaluate_clients,
        min_available_clients = num_total_clients,
        min_fit_clients = num_clients_per_round,
        min_evaluate_clients = num_evaluate_clients,
        evaluate_fn = federated_evaluation_function,
        on_fit_config_fn = lambda _: train_cfg,
        on_evaluate_config_fn = lambda _: test_cfg,
        initial_parameters = initial_parameters,
        fit_metrics_aggregation_fn = aggregate_weighted_average,
        evaluate_metrics_aggregation_fn = aggregate_weighted_average,
        federated_client_generator = local_federated_client_generator,
        server_learning_rate=server_learning_rate,
        server_momentum=server_momentum,
        accept_failures=accept_failures,
        target_accuracy=0.70,
        use_target_accuracy=True,
        )

    total_batch_results.append((batch_size, parameters_for_each_round, hist))
    save_experiment(f"federated_iid_batch_results_{batch_size}.pkl", batch_size, parameters_for_each_round, hist)

INFO flwr 2025-03-18 16:13:34,180 | flwr_core.py:107 | cid: 632
INFO flwr 2025-03-18 16:13:34,187 | flwr_core.py:107 | cid: 947
INFO flwr 2025-03-18 16:13:34,194 | flwr_core.py:107 | cid: 546
INFO flwr 2025-03-18 16:13:34,201 | flwr_core.py:107 | cid: 726
INFO flwr 2025-03-18 16:13:34,207 | flwr_core.py:107 | cid: 374
INFO flwr 2025-03-18 16:13:34,214 | flwr_core.py:107 | cid: 584
INFO flwr 2025-03-18 16:13:34,220 | flwr_core.py:107 | cid: 599
INFO flwr 2025-03-18 16:13:34,226 | flwr_core.py:107 | cid: 749
INFO flwr 2025-03-18 16:13:34,232 | flwr_core.py:107 | cid: 169
INFO flwr 2025-03-18 16:13:34,241 | flwr_core.py:107 | cid: 793
INFO flwr 2025-03-18 16:13:34,249 | flwr_core.py:107 | cid: 844
INFO flwr 2025-03-18 16:13:34,257 | flwr_core.py:107 | cid: 340
INFO flwr 2025-03-18 16:13:34,266 | flwr_core.py:107 | cid: 392
INFO flwr 2025-03-18 16:13:34,273 | flwr_core.py:107 | cid: 650
INFO flwr 2025-03-18 16:13:34,282 | flwr_core.py:107 | cid: 808
INFO flwr 2025-03-18 16:13:34,290 | flwr